In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'..')

In [55]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib
from pathlib import Path

In [56]:
data_path = Path('../food_dataset')

In [76]:
input_food = (pd.read_csv(data_path/'input_food.csv').set_index('fdc_id')[['gram_weight','sr_description']]
              .rename(columns = {'sr_description':'input','gram_weight':'input_weight'})
              .sort_values('input_weight'))

In [85]:
food = pd.read_csv(data_path/'food.csv')

In [86]:
food = food[food['data_type']=='survey_fndds_food'].set_index('fdc_id')[['description']]

In [87]:
food = food.join(input_food)

In [88]:
def func(g):
    g = g.sort_values('input_weight')
    g['cumsum'] = g['input_weight'].cumsum()/g['input_weight'].sum()
    g = g.sort_values('cumsum',ascending = False)
    
    g = g[g['cumsum']>0.05]
    return g

In [89]:
food = food.groupby('fdc_id').apply(func)

In [90]:
food = food.sort_values('input_weight',ascending = False)

In [92]:
food.index = [i[1] for i in food.index]

In [93]:
food.reset_index().groupby('index').

,description,input_weight,input,cumsum
1100629,"Bread, native, water, Puerto Rican style",4540.00,"Wheat flour, white, bread, enriched",1.000000
1100106,"Meat and corn hominy soup, home recipe, Mexica...",2970.00,"Soup, stock, chicken, home-prepared",1.000000
1100133,"Chicken or turkey and corn hominy soup, home ...",2970.00,"Soup, stock, chicken, home-prepared",1.000000
1097981,"Firni, Indian pudding",2928.00,"Milk, whole, 3.25% milkfat, with added vitamin D",1.000000
1099656,"Chicken or turkey creole, without rice",2880.00,"Chicken, NS as to part, rotisserie, skin not e...",1.000000
...,...,...,...,...
1102172,"Tortellini, cheese-filled, meatless, with toma...",1.40,"Onions, cooked, as ingredient",0.093280
1104254,"Coffee, instant, decaffeinated, pre-lightened ...",1.00,"Sweeteners, tabletop, sucralose, SPLENDA packets",0.155836
1102170,"Tortellini, meat-filled, with tomato sauce, ca...",0.70,"Beef, ground, 75% lean meat / 25% fat, crumble...",0.051043
1102170,"Tortellini, meat-filled, with tomato sauce, ca...",0.70,"Salt, table, iodized",0.058132


In [136]:
def func(g):
    g =g.sort_values('gram_weight',ascending=False)
    g = g[g['gram_weight'].cumsum()<g['gram_weight'].sum()*0.9]
    g['sr_description'] = ' ,'.join(g['sr_description'].tolist())
    
    return g.drop_duplicates('sr_description')[['sr_description','description']]
    

In [137]:
food_ = food.reset_index().groupby('fdc_id').apply(func)

In [138]:
f = food_.sample(1)

In [140]:
f['description'].iloc[0]

'Air filled fritter or fried puff, without syrup, Puerto Rican style'

In [139]:
f['sr_description'].iloc[0]

'Beverages, water, tap, drinking ,Egg, whole, raw, fresh ,Sugars, granulated ,Wheat flour, white, all-purpose, enriched, bleached'

In [142]:
food[food.index == f.reset_index()['fdc_id'][0]]

,description,gram_weight,sr_description
fdc_id,,,
1101164,"Air filled fritter or fried puff, without syru...",6.0,"Salt, table, iodized"
1101164,"Air filled fritter or fried puff, without syru...",28.0,"Vegetable oil, NFS"
1101164,"Air filled fritter or fried puff, without syru...",113.4,"Margarine, stick"
1101164,"Air filled fritter or fried puff, without syru...",125.0,"Wheat flour, white, all-purpose, enriched, ble..."
1101164,"Air filled fritter or fried puff, without syru...",200.0,"Egg, whole, raw, fresh"
1101164,"Air filled fritter or fried puff, without syru...",200.0,"Sugars, granulated"
1101164,"Air filled fritter or fried puff, without syru...",237.0,"Beverages, water, tap, drinking"


In [134]:
f.reset_index()

,fdc_id,level_1,sr_description,description
0,1102457,19214,"Salt, table, iodized ,Vegetable oil, NFS ,Gree...","Rice, brown, with other vegetables, NS as to fat"


In [131]:
food

,description,gram_weight,sr_description
fdc_id,,,
1097510,"Milk, human",100.0,"Milk, human, mature, fluid"
1097511,"Milk, NFS",11.0,"Milk, nonfat, fluid, with added vitamin A and ..."
1097511,"Milk, NFS",17.0,"Milk, lowfat, fluid, 1% milkfat, with added vi..."
1097511,"Milk, NFS",35.0,"Milk, whole, 3.25% milkfat, with added vitamin D"
1097511,"Milk, NFS",37.0,"Milk, reduced fat, fluid, 2% milkfat, with add..."
...,...,...,...
1104591,Sauce as ingredient in hamburgers,60.0,Catsup
1104592,Industrial oil as ingredient in food,25.0,"Oil, industrial, soy, fully hydrogenated"
1104592,Industrial oil as ingredient in food,25.0,"Oil, industrial, soy, low linolenic"


In [75]:
food = food.reset_index().groupby('fdc_id').agg({'description':'first','sr_description':lambda l:' ,'.join(list(l))})

In [77]:
food.sample(1)

,description,sr_description
fdc_id,,
1099526,"Chicken or turkey, potatoes, and vegetables ex...","Tomato products, canned, sauce ,Corn, sweet, y..."


In [46]:
food

,fdc_id,data_type,description,food_category_id,publication_date
559101,1097510,survey_fndds_food,"Milk, human",9602.0,2020-10-30
559102,1097511,survey_fndds_food,"Milk, NFS",1004.0,2020-10-30
559103,1097512,survey_fndds_food,"Milk, whole",1002.0,2020-10-30
559104,1097513,survey_fndds_food,"Milk, low sodium, whole",1002.0,2020-10-30
559105,1097514,survey_fndds_food,"Milk, calcium fortified, whole",1002.0,2020-10-30
...,...,...,...,...,...
566179,1104588,survey_fndds_food,Tomatoes as ingredient in omelet,9999.0,2020-10-30
566180,1104589,survey_fndds_food,Other vegetables as ingredient in omelet,9999.0,2020-10-30
566181,1104590,survey_fndds_food,Vegetables as ingredient in curry,9999.0,2020-10-30
566182,1104591,survey_fndds_food,Sauce as ingredient in hamburgers,9999.0,2020-10-30


In [33]:
food.index

Int64Index([1097510, 1097511, 1097512, 1097513, 1097514, 1097515, 1097516,
            1097517, 1097518, 1097519,
            ...
            1104583, 1104584, 1104585, 1104586, 1104587, 1104588, 1104589,
            1104590, 1104591, 1104592],
           dtype='int64', name='fdc_id', length=7083)